## [TODO]

- Dropout everywhere
- Should glove embeddings be updated?
- Dealing with unknown tokens
- You and stocknet-code have included y_T in calculation in ATA, while paper equations have not. Think about it
- Correct y_size to 2 from 1 because two classes *aye*, *no*

# Creating the Model

In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.distributions as ds

/home/ishans996/myProjects/rawnit_sawhney_defi_task/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

In [3]:
from collections import defaultdict

In [81]:
word_embed_size = 100
embedder_hidden_size = 100
gru_num_layers = 1
window_size = 4

dropout_vmd_in = 0.3
vmd_hidden_size = 150
g_size = 50
y_size = 1

In [46]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.Embedder = Embedder()
        self.VMD = VMD(self)
        self.ATA = ATA(self)
    
    
    def forward(self, X, Y):
        """
            
        """
        
        X = self.Embedder(X)
        self.VMD(X, Y)
        self.ATA()
#         print("X", X.shape)
#         print("kl", self.kl.shape)
#         print("g", self.g.shape)
#         print("y", self.y.shape)
        # result = something
#         return X  
    
    
    def get_z_and_z_distr(self, arg, is_prior):
        mean = nn.Linear(arg.size(-1), vmd_hidden_size)(arg)
        stddev = nn.Linear(arg.size(-1), vmd_hidden_size)(arg)      
        stddev = torch.sqrt(torch.exp(stddev))
        epsilon = torch.randn(vmd_hidden_size)
        
        z = mean if is_prior else mean + torch.mul(epsilon, stddev)
        z_pdf = ds.normal.Normal(loc=mean, scale=stddev)
        return z, z_pdf

In [47]:
class Embedder(nn.Module):
    def __init__(self):
        super(Embedder, self).__init__()
        self.bi_gru = nn.GRU(word_embed_size, embedder_hidden_size, num_layers=gru_num_layers, batch_first=True, bidirectional=True)
        

    def forward(self, X):
        """
            X: window_size (for speaker's history) * max_words_in_a_speech * input_size
            
        """
#         X = get_glove_padded_seq(X)
        h_0 = torch.randn(2*gru_num_layers, window_size, embedder_hidden_size) # multiplied by 2 because bidirectional
        _, h_n = self.bi_gru(X, h_0)
        h_f = h_n[0, :, :] 
        h_b = h_n[1, :, :]
        msg_embed = (h_f + h_b) / 2
        return msg_embed
    

In [48]:
class VMD(nn.Module):
    def __init__(self, model):
        super(VMD, self).__init__()
#         self.input_dropout = nn.Dropout(p=dropout_vmd_in)
        self.model = model    
        self.gru = nn.GRU(embedder_hidden_size, vmd_hidden_size, num_layers=gru_num_layers, bidirectional=False)
        
        
    def forward(self, X, Y):
#         X = self.input_dropout(X)
        h_0 = torch.randn(gru_num_layers, vmd_hidden_size) 
        h_s, _ = self.gru(X, h_0) # h_s: window_size * vmd_hidden_size
        
        z_prior = []
        z_post = []
        kl = []
        z_post_t_minus_1 = torch.randn(vmd_hidden_size)
        for t in range(window_size):
            h_z_prior_t = nn.Linear(embedder_hidden_size+2*vmd_hidden_size, vmd_hidden_size)(
                torch.cat([X[t], h_s[t], z_post_t_minus_1])
            )
            h_z_prior_t = nn.Tanh()(h_z_prior_t)

            h_z_post_t = nn.Linear(embedder_hidden_size+2*vmd_hidden_size+1, vmd_hidden_size)(
                torch.cat([X[t], h_s[t], Y[t][None], z_post_t_minus_1])
            )
            h_z_post_t = nn.Tanh()(h_z_post_t)
            
            z_prior_t, z_prior_t_pdf = self.model.get_z_and_z_distr(h_z_prior_t, is_prior=True)
            z_post_t, z_post_t_pdf = self.model.get_z_and_z_distr(h_z_post_t, is_prior=False)
            z_post_t_minus_1 = z_post_t
            
            kl_t = ds.kl.kl_divergence(z_prior_t_pdf, z_post_t_pdf)
            
            z_prior.append(z_prior_t)
            z_post.append(z_post_t)
            kl.append(kl_t)
        
        z_prior = torch.stack(z_prior) # window_size * vmd_hidden_size
        z_post = torch.stack(z_post) # window_size * vmd_hidden_size
        self.model.kl = torch.stack(kl).sum(dim=1) # window_size
        
        self.model.g = nn.Linear(2*vmd_hidden_size, g_size)(
            torch.cat([h_s, z_post],dim=1) # TODO: check if X is also to be concatenated as in Eqn 21
        )        
        self.model.g = nn.Tanh()(self.model.g) #
        
        self.model.y = nn.Linear(g_size, y_size)(self.model.g)
#         y = nn.Softmax(y)

        self.model.g_T = self.model.g[-1, :] 
        # TODO: 1. will g_T be different during training and evaluation?
        # TODO: 2. is g_T definitely what you think it is?


In [78]:
class ATA(nn.Module):
    def __init__(self, model):
        super(ATA, self).__init__()
        self.model = model
        self.alpha = 0.5

    def calculate_loss():
        v_aux = self.alpha * self.model.v_starred
        
        
    def forward(self):
        proj_i = nn.Linear(g_size, g_size, bias=False)(self.model.g)
        proj_i = nn.Tanh()(proj_i) # (window_size, g_size)
        w_i = nn.init.xavier_normal_(torch.zeros((g_size, 1))) # (g_size, 1)
        v_i = proj_i @ w_i # (window_size, 1)
        
        proj_d = nn.Linear(g_size, g_size, bias=False)(self.model.g)
        proj_d = nn.Tanh()(proj_d)
        g_T = self.model.g_T[:, None]
        v_d = proj_d @ g_T
        
        aux_score = torch.mul(v_i, v_d)
        aux_score[-1, :] = np.NINF
        
        v_starred = nn.Softmax(dim=0)(aux_score)
        self.model.v_starred = torch.where(torch.isnan(v_starred), torch.tensor(0, dtype=torch.float32), v_starred)
        
        att_c = self.model.y.T @ self.model.v_starred
        self.model.y_T = nn.Linear(1+g_size, y_size)(torch.cat([att_c, self.model.g_T]))
        self.model.y_T = nn.Softmax(dim=0)(self.model.y_T)  
        
        
        calculate_loss()

In [79]:
model = Model()
model(X, Y)

torch.Size([1, 4])


# Prepare train data

In [13]:
data = pd.read_csv("dataset/ParlVote/ParlVote_concat.csv")

In [14]:
# speakers[speaker_id: int] = list of tuples (debate_id, speech, vote)
speakers = defaultdict(lambda: [])
for idx, row in data.iterrows():
    speakers[row["speaker_id"]].append([row["debate_id"], row["speech"], row["vote"]])
        
for k, v in speakers.items():
    speakers[k] = sorted(v)

## Analysis of speaker sequence length

In [15]:
mx = 0
for usid in data.speaker_id.unique():
    mx = max(mx, len(speakers[usid]))
print(mx)

284


In [16]:
speaker_history_lens = []
for sid, each in speakers.items():
    speaker_history_lens.append(len(each)) 
speaker_history_lens.sort()

In [17]:
shl = pd.Series(speaker_history_lens)
shl.describe()

count    1346.000000
mean       24.859584
std        26.420400
min         1.000000
25%         8.000000
50%        16.000000
75%        32.000000
max       284.000000
dtype: float64

In [18]:
(len(list(filter(lambda x: x >= 4, speaker_history_lens))) / len(speaker_history_lens)) * 100

90.04457652303121

- The above cell shows 90% speakers have at least *window_size*=4 speeches
- If we use all data, we need to pad speeches that have (< *window_size*=4) speeches
- Decision: Drop these 10% speakers

In [19]:
speakers = dict(filter(lambda val: len(val[1]) >= 4, speakers.items()))

In [20]:
100 * (len(speakers) / len(speaker_history_lens))

90.04457652303121

## Download Glove-100d

In [21]:
# ! wget -r -nc https://nlp.stanford.edu/data/glove.6B.zip

In [22]:
# !unzip nlp.stanford.edu/data/glove.6B.zip

In [23]:
# ! head -n1 glove.6B.100d.txt

In [24]:
embeds = dict()
for line in open("nlp.stanford.edu/data/glove.6B.100d.txt", "r"):
    line = line.split()
    embeds[line[0]] = torch.tensor(list(map(float, line[1:])), dtype=torch.float32)

In [25]:
# glove_vocab = set()
# for f in open("nlp.stanford.edu/data/glove.6B.100d.txt", "r"):
#     glove_vocab.add(f.split()[0])
# print(len(glove_vocab))

# del glove_vocab

## Preprocess

In [26]:
from nltk.tokenize import TweetTokenizer

In [27]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [28]:
def preprocess(s: str):
    
    # Lowercasing
    s = s.lower()
    
    # Tokenization
    ttk = TweetTokenizer()
    tokens = ttk.tokenize(s)
#     tokens = word_tokenize(s)
    
    # Stopwords removal
    tokens = [w for w in tokens if not w in stopwords.words('english')]
    
#     # Stemming
#     ps = nltk.PorterStemmer()
#     tokens = [ps.stem(w) for w in tokens]
    return tokens

In [29]:
# %%timeit
# embed_keys = set(embeds.keys())
# ans = 0
# i = 0
# sids = speakers.keys()
# for sid in sids:
    
#     s = speakers[sid][0][1]
#     pre = preprocess(s)
#     ans += len(set(pre).difference((set(pre).intersection(embed_keys))))
    
#     print("i", i, " done:", ans)
#     i += 1

In [30]:
# embeds['abhorrent']

In [31]:
# input1 = embeds["drank"]
# input2 = embeds["drunk"]
# cos = nn.CosineSimilarity(dim=0)
# output = cos(input1, input2)

## Create PaddedTensor for input

In [32]:
from torch.nn.utils.rnn import pad_sequence

In [33]:
def get_X_Y_padded_tensors(debates):
        """
            X: window_size * max_seq_len * word_embed_size 
            Y: window_size
        """
        X = []
        Y = []
        max_tokens = 0
        for _, speech, vote in debates:
            tokens = preprocess(speech)
#             print("len preprocessed speech:", len(tokens))
            X.append(torch.stack([embeds.get(token, torch.randn(word_embed_size)) for token in tokens]))
            Y.append(vote)
        
        X = pad_sequence(X, batch_first=True)
        return X, torch.tensor(Y, dtype=torch.int32)

In [36]:
for speaker_id, debates in speakers.items():
    for i in range(window_size, len(debates)+1):
        X, Y = get_X_Y_padded_tensors(debates[i-window_size: i]) 
#         embedder_out = model.Embedder(X)
#         print(embedder_out.shape)
        break
    break